In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout,Conv2D, MaxPooling3D, Conv3D, MaxPooling2D, TimeDistributed,LSTM
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, Xception, InceptionV3,InceptionResNetV2



import tensorflow_addons as tfa




import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

#tensorflow 2.x
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
"""
#tensorflow 1.x
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
"""

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32



In [ ]:
num_classes = 4
img_size = (150,150)
X = np.load("../Data/Skin_Cancer/X_Skin_ISIC_Multi_150.npy")
Y = np.load("../Data/Skin_Cancer/Y_Skin_ISIC_Multi_150.npy")

In [ ]:
from random import sample

In [ ]:

index = [i for i in range(X.shape[0])]
np.random.shuffle(index) 


X2 = X[index]/255.0
Y2 = Y[index]

In [ ]:
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
label_encode = LabelEncoder()
Y3 = label_encode.fit_transform(Y2)

In [ ]:
Y3

In [ ]:
max(Y3)

In [ ]:
def activation_block(x):
    x = layers.Activation("gelu")(x)
    return layers.BatchNormalization()(x)


def conv_stem(x, filters: int, patch_size: int):
    x = layers.Conv2D(filters, kernel_size=patch_size, strides=patch_size)(x)
    return activation_block(x)


def conv_mixer_block(x, filters: int, kernel_size: int):
    # Depthwise convolution.
    x0 = x
    x = layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(x)
    x = layers.Add()([activation_block(x), x0])  # Residual.

    # Pointwise convolution.
    x = layers.Conv2D(filters, kernel_size=1)(x)
    x = activation_block(x)

    return x


def get_conv_mixer_256_8(
    image_size=150, filters=256, depth=8, kernel_size=5, patch_size=16, num_classes=4
):
    """ConvMixer-256/8: https://openreview.net/pdf?id=TVHS5Y4dNvM.
    The hyperparameter values are taken from the paper.
    """
    inputs = keras.Input((image_size, image_size, 3))
    #x = layers.Rescaling(scale=1.0 / 255)(inputs)

    # Extract patch embeddings.
    x = conv_stem(inputs, filters, patch_size)

    # ConvMixer blocks.
    for _ in range(depth):
        x = conv_mixer_block(x, filters, kernel_size)

    # Classification block.
    x = layers.GlobalAvgPool2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs, outputs)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score,confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=10)
earlystop = EarlyStopping(monitor='val_accuracy', patience=15)

In [ ]:
seed = 101
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvacc = []
cvauc = []
cvf1 = []

i = 1
for train, test in kfold.split(X2,Y2):
    model = get_conv_mixer_256_8()
    optimizer = tfa.optimizers.AdamW(
    learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    filepath = "../Data/models/150_ConvMix_{0}_skin_multi_16patch.h5".format(str(i))
    if os.path.exists(filepath):
        os.remove(filepath)
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1,monitor='val_accuracy',mode='max'
    )
    

    model.fit(X2[train], Y3[train], validation_split=0.2,epochs=300, batch_size=32, verbose=1,shuffle=True,callbacks=[checkpoint,earlystop])
    model.load_weights(filepath)
    # evaluate the model
    y_pred = model.predict(X2[test])
    acc = accuracy_score(Y3[test],[np.argmax(x) for x in y_pred])
    f1 = f1_score(Y3[test],[np.argmax(x) for x in y_pred],average='macro')
 
    auc = roc_auc_score(Y3[test],y_pred,multi_class='ovo')
    
    print("acc: %.2f%%" % (acc))
    print("auc: %.2f"%(auc))
    print("f1: %.2f"%(f1))
    print("confusion matrix:",confusion_matrix(Y3[test],[np.argmax(x) for x in y_pred]))
    
    cvacc.append(acc)
    cvauc.append(auc)
    cvf1.append(f1)

    i+=1
print("mean acc %.2f%% (+/- %.2f%%)" % (np.mean(cvacc), np.std(cvacc)))
print("mean auc %.2f%% (+/- %.2f%%)" % (np.mean(cvauc), np.std(cvauc)))
print("mean f1 %.2f%% (+/- %.2f%%)" % (np.mean(cvf1), np.std(cvf1)))


In [ ]:
cvacc

In [ ]:
np.mean(cvacc)

In [ ]:
cvauc

In [ ]:
np.mean(cvauc)

In [ ]:
cvf1

In [ ]:
np.mean(cvf1)